# Post process script with visualization

In [1]:
import sys
import matplotlib
sys.path.insert(0, '../')
from core import *
#config
from config import *
#CUDA
initCUDA(cuda)
#supporting files
from model import *
from helper import *
from matplotlib.ticker import FormatStrFormatter

matplotlib.pyplot.rcParams['font.family'] = 'serif'
matplotlib.pyplot.rcParams['mathtext.fontset'] = 'dejavuserif'

from post_process import *



-----------------------------------------------------
Setting device to:  0
Test:  cuda:0
-----------------------------------------------------


-----------------------------------------------------
Setting device to:  0
Test:  cuda:0
-----------------------------------------------------



/home/feolalab/anaconda3/envs/NNEUCLID/lib/python3.9/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608883701/work/torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [24]:
fem_material='HainesWilson'
noise_level='low'
output_dir='/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/VFM_compare/VFM/'

In [25]:
#Build architecture from config
model = ICNN(n_input=n_input,
                 n_hidden=n_hidden,
                 n_output=n_output,
                 use_dropout=use_dropout,
                 dropout_rate=dropout_rate)

In [26]:
#Evaluation based on losses 
final_losses = torch.zeros((ensemble_size,1))
for ensemble_iter in range(ensemble_size):
    final_losses[ensemble_iter] = pd.read_csv(output_dir+'/'+fem_material+'/loss_history_noise='+noise_level+'_ID='+str(ensemble_iter)+'.csv', header=None).values[-1][1]

final_losses_ratio = final_losses / torch.min(final_losses)
num_models_remove = torch.where(final_losses_ratio >= accept_ratio)[0].shape[0]
num_models_keep = ensemble_size - num_models_remove

idx_best_models = torch.topk(-final_losses.flatten(),num_models_keep).indices
idx_worst_models = torch.topk(final_losses.flatten(),num_models_remove).indices


In [27]:
final_losses, final_losses.mean(), final_losses.std()

(tensor([[38.0821],
         [29.8462],
         [30.2806],
         [30.5321],
         [30.9677],
         [29.9666],
         [30.6007],
         [30.5008],
         [30.2437],
         [30.1996],
         [30.1376],
         [30.5381],
         [29.9716],
         [30.3714],
         [30.1486]]),
 tensor(30.8258),
 tensor(2.0283))

In [11]:
#Get the true models
for path_count, strain_path in enumerate(strain_paths):
			if strain_path == 'UT':
				P_idx = 0
			elif strain_path == 'SS':
				P_idx = 1
			elif strain_path == 'PS':
				P_idx = 3
			elif strain_path == 'UC':
				P_idx = 0
			elif strain_path == 'BT':
				P_idx = 0
			elif strain_path == 'BC':
				P_idx = 0

gamma=np.linspace(g_min,g_max,gamma_steps)
F, xlabel = getStrainPathDeformationGradient(strain_path, gamma_steps, gamma)

# Get components of F
F11 = F[:,0:1]
F12 = F[:,1:2]
F21 = F[:,2:3]
F22 = F[:,3:4]

F11.requires_grad = True
F12.requires_grad = True
F21.requires_grad = True
F22.requires_grad = True

#computing detF
J = computeJacobian(torch.cat((F11,F12,F21,F22),dim=1))

#computing Cauchy-Green strain: C = F^T F
C = computeCauchyGreenStrain(torch.cat((F11,F12,F21,F22),dim=1))

#computing strain invariants
I1, I2, I3 = computeStrainInvariants(C)

#Get true model of fem_material
W_truth = get_true_W(fem_material,J,C,I1,I2,I3)

dW_truth_dF11 = torch.autograd.grad(W_truth,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]
dW_truth_dF12 = torch.autograd.grad(W_truth,F12,torch.ones(F12.shape[0],1),create_graph=True)[0]
dW_truth_dF21 = torch.autograd.grad(W_truth,F21,torch.ones(F21.shape[0],1),create_graph=True)[0]
dW_truth_dF22 = torch.autograd.grad(W_truth,F22,torch.ones(F22.shape[0],1),create_graph=True)[0]

P_truth = torch.cat((dW_truth_dF11,dW_truth_dF12,dW_truth_dF21,dW_truth_dF22),dim=1)


In [12]:
#Load models for inference:
W_predictions = torch.zeros((gamma_steps,ensemble_size))
P_predictions = torch.zeros((gamma_steps,4,ensemble_size))

for ensemble_iter in range(ensemble_size):
    model.load_state_dict(torch.load(output_dir+'/'+fem_material+'/noise='+noise_level+'_ID='+str(ensemble_iter)+'.pth'))


    W_predictions[:,ensemble_iter:ensemble_iter+1], P_predictions[:,:,ensemble_iter:ensemble_iter+1] = compute_corrected_W(F)

P_accepted = P_predictions[:,:,idx_best_models]
P_rejected = P_predictions[:,:,idx_worst_models]

W_accepted = W_predictions[:,idx_best_models]
W_rejected = W_predictions[:,idx_worst_models]

## Aux functions

In [9]:

def get_true_W(fem_material,J,C,I1,I2,I3):
    """

    Analytical description of benchmark hyperelastic material models.
    Input:		Strain invariants and Cauchy-Green deformation matrix.
    Output:		Strain energy density of the specified material

    """
    if fem_material == 'NeoHookean':
        I1_tilde = J**(-2/3)*I1
        W_truth = 0.5000*(I1_tilde - 3) + 1.5000*(J - 1)**2
    elif fem_material == 'Anisotropy45':
        I1_tilde = J**(-2/3)*I1
        alpha = torch.tensor([np.pi/4])
        C11 = C[:,0:1]
        C12 = C[:,1:2]
        C21 = C[:,2:3]
        C22 = C[:,3:4]
        Ia = torch.cos(alpha)*(C11*torch.cos(alpha)+C12*torch.sin(alpha)) + torch.sin(alpha)*(C21*torch.cos(alpha)+C22*torch.sin(alpha))
        Ia_tilde = Ia * J**(-2/3)
        W_truth = 0.5*(I1_tilde - 3.) + 0.75*(J - 1.)**2 + 0.5*(Ia_tilde - 1.)**2
    elif fem_material == 'Anisotropy60':
        I1_tilde = J**(-2/3)*I1
        alpha = torch.tensor([np.pi/3])
        C11 = C[:,0:1]
        C12 = C[:,1:2]
        C21 = C[:,2:3]
        C22 = C[:,3:4]
        Ia = torch.cos(alpha)*(C11*torch.cos(alpha)+C12*torch.sin(alpha)) + torch.sin(alpha)*(C21*torch.cos(alpha)+C22*torch.sin(alpha))
        Ia_tilde = Ia * J**(-2/3)
        W_truth = 0.5*(I1_tilde - 3.) + 0.75*(J - 1.)**2 + 0.5*(Ia_tilde - 1.)**2
    elif fem_material == 'Isihara':
        I1_tilde = J**(-2/3)*I1
        I2_tilde = J**(-4/3)*I2
        W_truth = 0.5000*(I1_tilde - 3) + 1.0000*(I2_tilde - 3) + 1.0000*(I1_tilde - 3)**2 + 1.5000*(J - 1)**2
    elif fem_material == 'HainesWilson':
        I1_tilde = J**(-2/3)*I1
        I2_tilde = J**(-4/3)*I2
        W_truth = 0.5000*(I1_tilde - 3) + 1.0000*(I2_tilde - 3) + 0.7000*(I1_tilde - 3)*(I2_tilde - 3) + 0.2000*(I1_tilde - 3)**3 + 1.5000*(J - 1)**2
    elif fem_material == 'GentThomas':
        I1_tilde = J**(-2/3)*I1
        I2_tilde = J**(-4/3)*I2
        W_truth = 0.5000*(I1_tilde - 3) + 1.5000*(J - 1)**2 + 1.0000*torch.log(I2_tilde/3)
    elif fem_material == 'ArrudaBoyce':
        shear_modulus = 2.5
        chain_length = 28
        kappa = 1.5
        shearModulus = torch.tensor([shear_modulus])
        N = torch.tensor([chain_length])
        sqrt_N = torch.sqrt(N)
        W_truth = torch.zeros((gamma_steps,1))
        I1_tilde = J**(-2/3)*I1
        lambda_chain = 1
        x = lambda_chain / torch.sqrt(N)
        beta_chain = 0.0
        if torch.abs(x)<0.841:
            beta_chain = 1.31*torch.tan(1.59*x)+0.91*x
        else:
            beta_chain = 1./((x+0.00000001/(torch.abs(x)+0.00000001))-x)
        W_truth_offset = shearModulus * sqrt_N * (beta_chain*lambda_chain+sqrt_N*torch.log(beta_chain/torch.sinh(beta_chain)))
        for step in range(gamma_steps):
            lambda_chain = torch.sqrt(I1_tilde[step:step+1]/3.)
            x = lambda_chain / torch.sqrt(N)
            beta_chain = 0.0
            if torch.abs(x)<0.841:
                beta_chain = 1.31*torch.tan(1.59*x)+0.91*x
            else:
                beta_chain = 1./((x+0.00000001/(torch.abs(x)+0.00000001))-x)
            #% Final output
            W_truth[step:step+1,:] = shearModulus * sqrt_N * (beta_chain*lambda_chain+sqrt_N*torch.log(beta_chain/torch.sinh(beta_chain))) - W_truth_offset + kappa*(J[step:step+1]-1)**2

    elif fem_material == 'Ogden':
        kappa_ogden = 1.5
        mu_ogden = 0.65
        alpha_ogden = 0.65
        I1_tilde = J**(-2/3)*I1 + 1e-13
        I1t_0 =torch.tensor([3]) + 1e-13
        J_0 = torch.tensor([1]) + 1e-13
        W_offset = kappa_ogden*(J_0-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1t_0  +  torch.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.)) - 1/(J_0**(2./3.)) )**alpha_ogden+( 0.5*I1t_0 - 0.5*torch.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.))  - 0.5/(J_0**(2./3.)) )**alpha_ogden + J_0**(-alpha_ogden*2./3.) ) * mu_ogden
        #% Final output
        W_truth = kappa_ogden*(J-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1_tilde  +  torch.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.)) - 1/(J**(2./3.)) )**alpha_ogden+( 0.5*I1_tilde - 0.5*torch.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.))  - 0.5/(J**(2./3.)) )**alpha_ogden + J**(-alpha_ogden*2./3.) ) * mu_ogden - W_offset

    elif fem_material == 'Holzapfel':
        I1_tilde = J**(-2/3)*I1
        alpha = torch.tensor([np.pi/6])
        beta = -1.*alpha
        C11 = C[:,0:1]
        C12 = C[:,1:2]
        C21 = C[:,2:3]
        C22 = C[:,3:4]
        Ia = torch.cos(alpha)*(C11*torch.cos(alpha)+C12*torch.sin(alpha)) + torch.sin(alpha)*(C21*torch.cos(alpha)+C22*torch.sin(alpha))
        Ib = torch.cos(beta)*(C11*torch.cos(beta)+C12*torch.sin(beta)) + torch.sin(beta)*(C21*torch.cos(beta)+C22*torch.sin(beta))
        Ia_tilde = Ia * J**(-2/3)
        Ib_tilde = Ib * J**(-2/3)
        k1h = 0.9
        k2h = 0.8
        HAa = k1h/2./k2h*(torch.exp(k2h*torch.pow(Ia_tilde - 1.,2.))-1.)
        HAb = k1h/2./k2h*(torch.exp(k2h*torch.pow(Ib_tilde - 1.,2.))-1.)
        #% Final output
        W_truth = 0.5*(I1_tilde-3) + 1.0*(J-1)**2 + HAa + HAb

    return W_truth

In [10]:
def compute_corrected_W(F):
    """
    Compute the strain energy density according to Ansatz (Eq. 8).

    Input: 		Deformation gradient F in form (F11,F12,F21,F22)
    Output: 	Strain energy density according to Ansatz (Eq. 8)

    """

    F_0 = torch.zeros((1,4))
    F_0[:,0] = 1
    F_0[:,3] = 1

    F11_0 = F_0[:,0:1]
    F12_0 = F_0[:,1:2]
    F21_0 = F_0[:,2:3]
    F22_0 = F_0[:,3:4]

    F11_0.requires_grad = True
    F12_0.requires_grad = True
    F21_0.requires_grad = True
    F22_0.requires_grad = True

    # Get components of F
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]

    F11.requires_grad = True
    F12.requires_grad = True
    F21.requires_grad = True
    F22.requires_grad = True

    W_NN = model(torch.cat((F11,F12,F21,F22),dim=1))

    dW_NN_dF11 = torch.autograd.grad(W_NN,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]
    dW_NN_dF12 = torch.autograd.grad(W_NN,F12,torch.ones(F12.shape[0],1),create_graph=True)[0]
    dW_NN_dF21 = torch.autograd.grad(W_NN,F21,torch.ones(F21.shape[0],1),create_graph=True)[0]
    dW_NN_dF22 = torch.autograd.grad(W_NN,F22,torch.ones(F22.shape[0],1),create_graph=True)[0]

    P_NN = torch.cat((dW_NN_dF11,dW_NN_dF12,dW_NN_dF21,dW_NN_dF22),dim=1)

    # Derivative of volumetric term
    J_F_inv_T = torch.cat((F22,-F21,-F12,F11),1)
    C = computeCauchyGreenStrain(torch.cat((F11,F12,F21,F22),dim=1))
    _,_,I3 = computeStrainInvariants(C)

    # Zero deformation input data
    W_NN_0 = model(torch.cat((F11_0,F12_0,F21_0,F22_0),dim=1))

    dW_NN_dF11_0 = torch.autograd.grad(W_NN_0,F11_0,torch.ones(F11_0.shape[0],1),create_graph=True)[0]
    dW_NN_dF12_0 = torch.autograd.grad(W_NN_0,F12_0,torch.ones(F12_0.shape[0],1),create_graph=True)[0]
    dW_NN_dF21_0 = torch.autograd.grad(W_NN_0,F21_0,torch.ones(F21_0.shape[0],1),create_graph=True)[0]
    dW_NN_dF22_0 = torch.autograd.grad(W_NN_0,F22_0,torch.ones(F22_0.shape[0],1),create_graph=True)[0]

    P_NN_0 = torch.cat((dW_NN_dF11_0,dW_NN_dF12_0,dW_NN_dF21_0,dW_NN_dF22_0),dim=1)

    P_cor = torch.zeros_like(P_NN)

    P_cor[:,0:1] = F11*-P_NN_0[:,0:1] + F12*-P_NN_0[:,2:3]
    P_cor[:,1:2] = F11*-P_NN_0[:,1:2] + F12*-P_NN_0[:,3:4]
    P_cor[:,2:3] = F21*-P_NN_0[:,0:1] + F22*-P_NN_0[:,2:3]
    P_cor[:,3:4] = F21*-P_NN_0[:,1:2] + F22*-P_NN_0[:,3:4]

    P = P_NN + P_cor
    E = torch.zeros_like(F)

    E[:,0:1] = 0.5*(C[:,0:1]-1.)
    E[:,1:2] = 0.5*(C[:,1:2])
    E[:,2:3] = 0.5*(C[:,2:3])
    E[:,3:4] = 0.5*(C[:,3:4]-1.)

    W_cor = torch.sum(-P_NN_0*E, 1, keepdim=True)

    # The actual offset of the NN output is W_NN_0 (NN ouput at F=I) and H:E
    W_offset = W_NN_0
    # W is sum of NN output, the volumetric correction term and subtracting NN output at F=I and H:E
    W = W_NN + W_cor - W_offset

    P = P.view(-1,4,1)

    return W, P